In [ ]:
%matplotlib inline

from yapss import Problem
import numpy as np
import matplotlib.pyplot as plt
import scipy

## Acrobot Parameters
These parameters describe the physical acrobot and are as defined in [1]. Additionally, we define gravity as $9.8\: \mathrm{m/s^2}$.

In [ ]:
### Acrobot parameters (taken from Spong 1995)
# Masses (kg)
m1 = 1.
m2 = 1.
# Link lengths (m)
l1 = 1.
l2 = 2.
# Centers of masses (m)
lc1 = l1/2.
lc2 = l2/2.
# Moments of inertia
I1 = 0.083
I2 = 0.33
# Gravity
g = 9.8

In [ ]:
### Acrobot dynamics & objective function

# Simple helper function
def inv_2x2(A):
    """Returns inverse of 2x2 matrix A"""
    assert A.shape == (2, 2), "Wrong shape for matrix A, cannot invert"

    a11 = A[0,0]
    a12 = A[0, 1]
    a21 = A[1, 0]
    a22 = A[1,1]

    det = a22 * a11 - a12 * a21
    assert det != 0, "Singular matrix!"
    return np.array([[a22/det, -a12/det],
                     [-a21/det, a11/det]])

def dynamics_tedrake(q1, q2, q1_dot, q2_dot, u, b1=0., b2=0.):
    # M(q)*q_ddot + C(q, q_dot) = tau_g(q) + Bu
    # q = [q1, q2]^T
    # B = [0, 1]^T
    # Source: https://underactuated.csail.mit.edu/acrobot.html#section1,
    # but with mass matrix from DOI:10.1016/j.ifacol.2016.10.194
    # Coriolis and gravity terms match between sources (accounting for from axes conventions)

    a1 = m1*lc1**2 + m2*l1**2 + I1
    a2 = m2*lc2**2 + I2
    a3 = m2*l1*lc2

    # M, mass matrix
    # M = np.array([[I1 + I2 + m2*l1**2 + 2*m2*l1*lc2*np.cos(q2), I2 + m2*l1*lc2*np.cos(q2)],
    #               [I2 + m2*l1*lc2*np.cos(q2), I2]])  # this is from Tedrake and is wrong
    M = np.array([[a1 + a2 + 2*a3*np.cos(q2), a2 + a3*np.cos(q2)],
                  [a2 + a3*np.cos(q2), a2]])

    # C, Coriolis terms
    C = np.array([-2*a3*np.sin(q2)*q2_dot*q1_dot - a3*np.sin(q2)*q2_dot**2,
                  a3*np.sin(q2)*q1_dot**2])

    # tau_g, gravity terms
    tau_g = np.array([-m1*g*lc1*np.sin(q1) - m2*g*(l1*np.sin(q1) + lc2*np.sin(q1 + q2)),
                      -m2*g*lc2*np.sin(q1 + q2)])
    
    # Damping terms
    tau_g -= np.array([b1 * q1_dot, b2 * q2_dot])

    q_ddot = inv_2x2(M) @ (-C + tau_g + np.array([0, u])).reshape((2,1))
    return [q1_dot, q2_dot, q_ddot[0,0], q_ddot[1,0]]

def dynamics_spong(q1, q2, q1_dot, q2_dot, u):
    """
    Computes and returns $\ddot{q}$ as defined in:
    Mark Spong, "The Swingup Control Problem for the Acrobot", IEEE Control Systems
    Magazine, vol. 15, no. 1, pp. 49-55, February, 1995.

    This function uses the exact variable naming and angle/axis conventions as in the paper.
    """
    d11 = m1*lc1**2 + m2*(l1**2 + lc2**2 + 2*l1*lc2*np.cos(q2)) + I1 + I2
    d22 = m2*lc2**2 + I2
    d12 = m2*(lc2**2 + l1*lc2*np.cos(q2)) + I2
    d21 = m2*(lc2**2 + l1*lc2*np.cos(q2)) + I2
    h1 = -m2*l1*lc2*np.sin(q2)*q2_dot**2 - 2*m2*l1*lc2*np.sin(q2)*q2_dot*q1_dot
    h2 = m2*l1*lc2*np.sin(q2)*q1_dot**2
    phi1 = (m1*lc1 + m2*l1)*g*np.cos(q1) + m2*lc2*g*np.cos(q1 + q2)
    phi2 = m2*lc2*g*np.cos(q1 + q2)

    det = d22 * d11 - d12 * d21
    d_inv = np.array([[d22/det, -d12/det],
                      [-d21/det, d11/det]])
    q_ddot = d_inv @ np.array([-h1 - phi1, -h2 - phi2 + u]).reshape((2,1))

    return [q1_dot, q2_dot, q_ddot[0,0], q_ddot[1,0]]

def cost_u_only(Q, R, q1, q2, q1_dot, q2_dot, u):
    return R * u**2

In [ ]:
### Helper plotting functions
def plot_state(time, q1, q2, q1_dot, q2_dot, title=None):
    # Plot thetas and theta_dots
    fig = plt.figure(figsize=(12, 4))
    if title is not None:
        fig.suptitle(title)
    ax1 = fig.add_subplot(121)
    ax1.plot(time, q1, label="$q_1$")
    ax1.plot(time, q2, label="$q_2$")
    ax1.set_xlabel("Time (s)")
    ax1.set_ylabel("Angle (rad)")
    plt.grid()
    plt.legend()

    ax2 = fig.add_subplot(122)
    ax2.plot(time, q1_dot, label="$\\dot{q}_1$")
    ax2.plot(time, q2_dot, label="$\\dot{q}_2$")
    ax2.set_xlabel("Time (s)")
    ax2.set_ylabel("Angular velocity (rad/s)")
    plt.grid()
    plt.legend()

def plot_input(time, u):
    fig = plt.figure()
    plt.plot(time, u, label="$\\tau$")
    fig.suptitle("Optimal input")
    plt.xlabel("Time (s)")
    plt.ylabel("Torque (N-m)")
    plt.grid()

In [ ]:
### Problem definition
## Wrapped in a function to easily do multiple runs with different parameters
def acrobot_swing_up(dynamics_fn, cost_fn, u_max, t0, tf, x0, Q=1, R=10,
                     xf=None, x_max=None, state_guess=None, time_guess=None,
                     b1=0., b2=0.):
    def objective(arg):
        arg.objective = arg.phase[0].integral[0]

    def continuous(arg):
        q1, q2, q1_dot, q2_dot = arg.phase[0].state
        u, = arg.phase[0].control  # elbow torque
        arg.phase[0].dynamics[:] = dynamics_fn(q1, q2, q1_dot, q2_dot, u)
        arg.phase[0].integrand[0] = cost_fn(Q, R, q1, q2, q1_dot, q2_dot, u)

    # Acrobot state: [theta1, theta2, theta1_dot, theta2_dot]
    # input: [torque] (at elbow joint only)
    problem = Problem(name="acrobot", nx=[4], nu=[1], nq=[1])

    functions = problem.functions
    functions.objective = objective
    functions.continuous = continuous

    ## Set bounds
    bounds = problem.bounds.phase[0]

    # Initial conditions
    bounds.initial_time.lower = bounds.initial_time.upper = t0
    bounds.initial_state.lower = bounds.initial_state.upper = x0

    if xf is not None:
        bounds.final_state.lower = bounds.final_state.upper = xf
    
    # Final time
    bounds.final_time.lower = bounds.final_time.upper = tf

    # Control bounds (torque, N*m)
    bounds.control.lower[:] = (-u_max,)
    bounds.control.upper[:] = (u_max,)

    # State bounds
    if x_max is not None:
        bounds.state.lower = -1 * np.array(x_max)
        bounds.state.upper = x_max
    
    ## Compute initial guess if not given
    if state_guess is None and time_guess is None:
        if xf is None:  # assume this means passive dynamics
            sol = scipy.integrate.solve_ivp(lambda t,y: dynamics_fn(*y, 0, b1=b1, b2=b2), [t0, tf], x0)
            state_guess = [sol.y[0], sol.y[1], sol.y[2], sol.y[3]]
            time_guess = sol.t
        else:
            # Simulate the swing-up backwards
            sol = scipy.integrate.solve_ivp(lambda t,y: dynamics_fn(*y, 0, b1=b1, b2=b2),
                                            [t0, tf-0.05],
                                            [xf[0] - 0.02, xf[1], xf[2], xf[3]])
            state_guess = [np.flip(sol.y[0]), np.flip(sol.y[1]),
                        np.flip(-1*sol.y[2]), np.flip(-1*sol.y[3])]
            # Extra time step with final state--see if it holds it up there
            state_guess = [state_guess[0].tolist() + [xf[0]], state_guess[1].tolist()+ [xf[1]], state_guess[2].tolist()+ [xf[2]], state_guess[3].tolist()+ [xf[3]]]
            time_guess = sol.t.tolist() + [tf]

    ## Set initial guess
    initial_guess = problem.guess.phase[0]
    initial_guess.time = time_guess
    initial_guess.state = state_guess

    ## Mesh and solver settings
    # Prof. Hall suggested 3-400 points here
    m, n = 40, 10
    problem.mesh.phase[0].collocation_points = m * (n,)
    problem.mesh.phase[0].mesh_fraction = m * (1.0 / m,)

    problem.derivatives.order = "second"
    problem.derivatives.method = "auto"
    problem.spectral_method = "lgl"
    problem.ipopt_options.tol = 1e-8
    problem.ipopt_options.max_iter = 1000

    ## Plot the initial guess
    time = time_guess
    q1, q2, q1_dot, q2_dot = state_guess
    plot_state(time, q1, q2, q1_dot, q2_dot, title="Initial state trajectory guess")

    ## Solve the optimization
    output = problem.solve()

    # Extract information from solution
    time = output.solution.phase[0].time
    time_u = output.solution.phase[0].time_u
    q1, q2, q1_dot, q2_dot = output.solution.phase[0].state
    (torque,) = output.solution.phase[0].control
    return (time, q1, q2, q1_dot, q2_dot, time_u, torque)

In [ ]:
### Animate solution
# Adapted from: https://jckantor.github.io/CBE30338/A.03-Animation-in-Jupyter-Notebooks.html
from matplotlib import patches, animation, rc
rc("animation", html="html5")
from functools import partial

## Animation functions
# Must return parts of the frame that have changed in order to use blit
def drawframe_tedrake(n_frame, q1, q2, link1, link2):
    x1 = l1 * np.sin(q1[n_frame])
    y1 = -l1 * np.cos(q1[n_frame])
    link1.set_data([0, x1], [0, y1])
    x2 = x1 + l2 * np.sin(q1[n_frame] + q2[n_frame])
    y2 = y1 - l2 * np.cos(q1[n_frame] + q2[n_frame])
    link2.set_data([x1, x2], [y1, y2])
    return (link1, link2)

def drawframe_spong(n_frame, q1, q2, link1, link2):
    x1 = l1 * np.cos(q1[n_frame])
    y1 = l1 * np.sin(q1[n_frame])
    link1.set_data([0, x1], [0, y1])
    x2 = x1 + l2 * np.cos(q1[n_frame] + q2[n_frame])
    y2 = y1 + l2 * np.sin(q1[n_frame] + q2[n_frame])
    link2.set_data([x1, x2], [y1, y2])
    return (link1, link2)

## Wrapper function
def animate(q1, q2, drawframe, dt_sec=0.01):
    fig = plt.figure(figsize=(6,6))
    ax1 = plt.subplot(111)
    ax1.set_xlim((-4, 4))
    ax1.set_ylim((-4, 4))
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)

    # Shoulder joint doesn't move
    ax1.add_patch(patches.Circle((0, 0), 0.1, facecolor='lime', edgecolor='black'))

    ## Create objects that will change in the animation
    # Initially empty, but will be given new values for each frame in the animation
    link1, = ax1.plot([], [], 'r', lw=3)  # ax.plot returns a list of 2D line objects
    link2, = ax1.plot([], [], 'b', lw=3)

    ## Create the animation object
    # interval is in ms
    # blit=True redraws only the parts that have changed
    # Have to animate with constant dt's even though the solution is given with varied dt's
    anim = animation.FuncAnimation(fig, partial(drawframe, q1=q1, q2=q2, link1=link1, link2=link2), frames=len(q1), interval=dt_sec*1e3, blit=True)
    return anim

# Cases

Below, we step through some different boundary conditions and initial guesses to see how the optimal swing-up solution differs.

In [ ]:


### Problem parameters
t0 = 0
tf = 5

## Passive dynamics from horizontal

In [ ]:
u_max = 0
t0 = 0
tf = 5.
x0 = [-np.pi/2, 0, 0, 0]

time, q1, q2, q1_dot, q2_dot, _, _ = acrobot_swing_up(dynamics_tedrake, lambda *args: 0,
                                                      u_max, t0, tf, x0)

plot_state(time, q1, q2, q1_dot, q2_dot, title="Optimal trajectory")
anim = animate(q1, q2, drawframe_tedrake, dt_sec=tf/len(time))
anim

## Swing-up from stable fixed point

The downward position of the acrobot (both links straight down in line with gravity) is a stable fixed point. We want to swing the acrobot up from this position to the unstable fixed point at the top (both links still in line with gravity, but pointing up).

### Use coarse known solution as initial guess

To start, we can try seeding the solver with just a few points that coarsely specify a swing-up solution. This known solution comes from [3]. Unsurprisingly, the solver converges pretty quickly to a simple swing-up trajectory.

In [ ]:
u_max = 8  # N-m
t0 = 0
tf = 4.
# x0 = [-np.pi/2, 0, 0, 0]
x0 = [0, 0, 0, 0]
xf = [np.pi, 0, 0, 0]
R = 10  # input cost

# Source: https://underactuated.csail.mit.edu/trajopt.html#section3
KNOWN_SOLUTION_TIME = [0.0, 0.2, 1.4, 2.0, 4.0]
KNOWN_SOLUTION_STATE = [[x0[0], 0.35, -1.19, 1.12, xf[0]],
                        [x0[1], -0.71, 0.77, 2.48, xf[1]],
                        [x0[2], 2.63, -6.87, 5.99, xf[2]],
                        [x0[3], -5.34, 15.17, -3.22, xf[3]]]

time, q1, q2, q1_dot, q2_dot, time_u, torque = acrobot_swing_up(dynamics_tedrake, cost_u_only,
                                                                u_max, t0, tf, x0, R=R, xf=xf,
                                                                state_guess=KNOWN_SOLUTION_STATE,
                                                                time_guess=KNOWN_SOLUTION_TIME)
plot_state(time, q1, q2, q1_dot, q2_dot, title="Optimal trajectory")
plot_input(time_u, torque)
anim = animate(q1, q2, drawframe_tedrake, dt_sec=tf/len(time))
anim

### Use reversed damped falling dynamics as initial guess
What if we don't have a known trajectory handy? Instead, as an initial guess, we can simulate the dynamics of the acrobot falling from the unstable fixed point with damping at the joints and seed our solver with the reverse of these dynamics. We do still need to have an idea of how long the swing-up will take, and we need our damping to sufficiently mirror our torque limits (that is, a very low torque limit should correspond to less damping).

In [ ]:
u_max = 8  # N-m
t0 = 0
tf = 4
x0 = [0, 0, 0, 0]
xf = [np.pi, 0, 0, 0]
R = 10  # input cost

# Damping coefficients
b1 = 2

time, q1, q2, q1_dot, q2_dot, time_u, torque = acrobot_swing_up(dynamics_tedrake, cost_u_only,
                                                                u_max, t0, tf, x0, R=R, xf=xf)
plot_state(time, q1, q2, q1_dot, q2_dot, title="Optimal trajectory")
plot_input(time_u, torque)
anim = animate(q1, q2, drawframe_tedrake, dt_sec=tf/len(time))
anim

This trajectory works as well! But perhaps our acrobot cannot physically have the lower link rotate all the way around, so we need to impose state constraints on the problem. We can restrict $q_2$ to be within $(-\pi, \pi)$ and put very loose bounds on the remaining states (bounds which shouldn't be active anyway).

In [ ]:
# TODO can't get this one to converge now either

u_max = 6  # N-m
x_max = [10*np.pi, np.pi, 100, 100]
t0 = 0
tf = 4
x0 = [0, 0, 0, 0]
xf = [np.pi, 0, 0, 0]
R = 10  # input cost

# Damping coefficients
b1 = 2

time, q1, q2, q1_dot, q2_dot, time_u, torque = acrobot_swing_up(dynamics_tedrake, cost_u_only,
                                                                u_max, t0, tf, x0, R=R, xf=xf,
                                                                x_max=x_max)
plot_state(time, q1, q2, q1_dot, q2_dot, title="Optimal trajectory")
plot_input(time_u, torque)
anim = animate(q1, q2, drawframe_tedrake, dt_sec=tf/len(time))
anim

### Restrict the maximum torque allowed
As we reduce the maximum torque allowed, the acrobot must swing more times to add sufficient energy to reach the upward position. To account for this, the final trajectory time must also be increased. We can see below that we can cut the torque limit in half, increase the final time by 1 second, and use the same initial guess to get a valid swing-up solution.

In [ ]:
# Damped passive dynamics
u_max = 0
t0 = 0
tf = 4.
x0 = [np.pi - 0.02, 0, 0, 0]
b1 = 4
b2 = 5

time, q1, q2, q1_dot, q2_dot, _, _ = acrobot_swing_up(dynamics_tedrake, lambda *args: 0,
                                                      u_max, t0, tf, x0, b1=b1, b2=b2)

plot_state(time, q1, q2, q1_dot, q2_dot, title="Optimal trajectory")
anim = animate(q1, q2, drawframe_tedrake, dt_sec=tf/len(time))
anim

In [ ]:
u_max = 8  # N-m
t0 = 0
tf = 4
x0 = [0, 0, 0, 0]
xf = [np.pi, 0, 0, 0]
R = 10  # input cost

# Damping coefficients
b1 = 2

time, q1, q2, q1_dot, q2_dot, time_u, torque = acrobot_swing_up(dynamics_tedrake, cost_u_only,
                                                                u_max, t0, tf, x0, R=R, xf=xf)
plot_state(time, q1, q2, q1_dot, q2_dot, title="Optimal trajectory")
plot_input(time_u, torque)
anim = animate(q1, q2, drawframe_tedrake, dt_sec=tf/len(time))
anim

In [ ]:
u_max = 4  # N-m
tf = 5.

time, q1, q2, q1_dot, q2_dot, time_u, torque = acrobot_swing_up(dynamics_tedrake, cost_u_only,
                                                                u_max, t0, tf, x0, R=R, xf=xf,
                                                                state_guess=KNOWN_SOLUTION_STATE,
                                                                time_guess=KNOWN_SOLUTION_TIME)
plot_state(time, q1, q2, q1_dot, q2_dot, title="Optimal trajectory")
plot_input(time_u, torque)
anim = animate(q1, q2, drawframe_tedrake, dt_sec=tf/len(time))
anim

In [ ]:
# TODO haven't solved this yet with throttled torque
u_max = 2  # N-m
tf = 6.

time, q1, q2, q1_dot, q2_dot, time_u, torque = acrobot_swing_up(dynamics_tedrake, cost_u_only,
                                                                u_max, t0, tf, x0, R=R, xf=xf,
                                                                state_guess=KNOWN_SOLUTION_STATE,
                                                                time_guess=KNOWN_SOLUTION_TIME)
plot_state(time, q1, q2, q1_dot, q2_dot, title="Optimal trajectory")
plot_input(time_u, torque)
anim = animate(q1, q2, drawframe_tedrake, dt_sec=tf/len(time))
anim

# References
[1] Mark Spong, "The Swingup Control Problem for the Acrobot", IEEE Control Systems Magazine, vol. 15, no. 1, pp. 49-55, February, 1995.  
[2]  
[3] Russ Tedrake. Underactuated Robotics: Algorithms for Walking, Running, Swimming, Flying, and Manipulation (Course Notes for MIT 6.832). Downloaded on April 29, 2024 from https://underactuated.csail.mit.edu/.